# Install Required Libraries
In this section, we install the required libraries for Generative AI, focusing on image generation using Stable Diffusion.


In [ ]:
!pip install diffusers transformers accelerate


# Introduction to Generative AI
Generative AI involves creating new content, such as images, text, or audio, using AI models. For image generation:
- **Generator**: Creates new images based on input (e.g., a text prompt).
- **Discriminator**: Evaluates the generated content (mainly used in GANs).

In this notebook, we'll use a pre-trained Stable Diffusion model to generate images from text prompts.


# Import Libraries
Import the necessary libraries for using the Stable Diffusion model.


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image


# Load a Pre-trained Model
Here we load a pre-trained Stable Diffusion model from the `diffusers` library. This model generates images based on a given text prompt.


In [ ]:
# Load the Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

print("Model loaded successfully!")


# Generate an Image
Provide a text prompt to the Stable Diffusion model to generate an image.


In [ ]:
# Text prompt
prompt = input("Enter prompt : ")

# Generate the image
image = pipe(prompt).images[0]

# Display the image
image.show()


Enter prompt : A couple is walking in a snow valley in sunsine weather


  0%|          | 0/50 [00:00<?, ?it/s]

# Save the Generated Image
Save the generated image to the local directory for future use.


In [ ]:
# Save the image
image.save("generated_image7.png")
print("Image saved as 'generated_image7.png'")


Image saved as 'generated_image7.png'


# Generator and Discriminator Concepts
- **Generator**: In this case, the Stable Diffusion model acts as the generator, transforming a text prompt into an image.
- **Discriminator**: While not explicitly used here, in other models like GANs (Generative Adversarial Networks), the discriminator evaluates the quality of generated images.

This distinction is important in understanding how Generative AI works:
1. The generator creates content.
2. The discriminator evaluates and provides feedback (if applicable).


# Install Libraries for Discriminator
We need `torchvision` to use a pre-trained ResNet-50 model for evaluating or classifying the generated image.


In [ ]:
!pip install torchvision


# Load a Pre-trained Discriminator
Here, we load a ResNet-50 model pre-trained on ImageNet. This model acts as the discriminator to classify the generated image.


In [ ]:
import torchvision.transforms as transforms
from torchvision.models import resnet50
import torch.nn as nn

# Load ResNet-50 pre-trained on ImageNet
discriminator = resnet50(pretrained=True)
discriminator.eval()

# Transformation for input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224 for ResNet
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize(           # Normalize using ImageNet mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("Discriminator (ResNet-50) loaded successfully!")


# Evaluate the Generated Image
We use the pre-trained ResNet-50 to classify the generated image and output the top predictions. This simulates how a discriminator evaluates or labels the generated image.


In [ ]:
# Convert PIL image to tensor for ResNet
input_image = transform(image).unsqueeze(0)  # Add batch dimension

# Pass the image through the discriminator
with torch.no_grad():
    output = discriminator(input_image)

# Load ImageNet class labels
!wget -qO imagenet_classes.txt https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json
import json
with open("imagenet_classes.txt", "r") as f:
    labels = json.load(f)

# Get top 5 predictions
probabilities = torch.nn.functional.softmax(output[0], dim=0)
top5_prob, top5_catid = torch.topk(probabilities, 5)

print("Top 5 Predictions for the Generated Image:")
for i in range(top5_prob.size(0)):
    print(f"{labels[top5_catid[i]]}: {top5_prob[i].item():.4f}")
